# Evaluating a NeMo checkpoint with simple-evals

This notebook demonstrates how to extend the evaluation capabilities within the NeMo Framework container.
It guides you through installing an additional evaluation harness and various methods for specifying benchmarks.

For a better understanding of in-framework deployment and the distinction between completion and chat endpoints, please refer to the tutorial on ["Evaluating a NeMo checkpoint with lm-eval"](mmlu.ipynb) first.

In this tutorial, we will evaluate an LLM on the [HumanEval benchmark](https://arxiv.org/abs/2107.03374) available in [NVIDIA Evals Factory simple-evals](https://pypi.org/project/nvidia-simple-evals/) package.
HumanEval comprises 164 manually created Python programming problems, each defined by a function signature and a docstring explaining its purpose.
This benchmark assesses the functional correctness of generated code by comparing it against unit tests, rather than relying on textual similarity to a reference solution.

We will utilize the chat variant of the benchmark, specifically designed for evaluating the coding abilities of instruction-tuned (chat) models.

> Note: It is recommended to run this notebook within a [NeMo Framework container](https://catalog.ngc.nvidia.com/orgs/nvidia/containers/nemo), as it includes all necessary dependencies.

In [ ]:
import importlib
import logging
import signal
import subprocess

import core_evals
from nemo_evaluator.api.api_dataclasses import (
    ApiEndpoint,
    EndpointType,
    EvaluationConfig,
    EvaluationTarget,
)
from nemo_evaluator.core.evaluate import evaluate
from nemo_evaluator.core.input import get_available_evaluations

logging.getLogger().setLevel(logging.INFO)

## 1. Adding evaluation harness

Let's begin by exploring the available evaluations.

First, we'll examine the benchmarks that come pre-installed with the [NeMo Framework container](https://catalog.ngc.nvidia.com/orgs/nvidia/containers/nemo).

The function `get_available_evaluations` identifies all tasks across all installed evaluation frameworks.
Initially, it will only return `lm_evaluation_harness`.

Note that by default, we can find `mmlu` but we cannot locate a framework for executing `humaneval`.

In [ ]:
framwork_mapping, _, task_mapping = get_available_evaluations()

print("frameworks:", framwork_mapping.keys())
for task in ("mmlu", "humaneval"):
    try:
        task_evaluation = task_mapping[task]
        print(f"{task} found in {task_evaluation.framework_name}")
    except Exception as e:
        print("Error:", e)

Now, we will install an additional evaluation framework: [NVIDIA Evals Factory simple-evals](https://pypi.org/project/nvidia-simple-evals/).

It can be added by simply installing the package using `pip`.

We will then reload the `core_evals` module to ensure the changes are reflected.

In [ ]:
! pip install -q nvidia-simple-evals
importlib.reload(core_evals)

If we repeat the same checks as before, we can now see the newly installed framework and find an implementation for the `humaneval` task.

At the same time, since both `lm-evaluation-harness` and `simple-evals` implement `mmlu`, we need to specify the version of this task if we want to execute it.

In [ ]:
framwork_mapping, _, task_mapping = get_available_evaluations()

print("frameworks:", framwork_mapping.keys())
for task in ("mmlu", "humaneval"):
    try:
        task_evaluation = task_mapping[task]
        print(f"{task} found in {task_evaluation.framework_name}")
    except Exception as e:
        print("Error:", e)

## 2. Deploying the model

We are now ready to deploy and evaluate the model.

First, you need to prepare a NeMo 2 checkpoint of the model you would like to evaluate.
For the purpose of this tutorial, we will use the Llama 3.2 1B Instruct checkpoint, which you can download from the [NGC Catalog](https://catalog.ngc.nvidia.com/orgs/nvidia/teams/nemo/models/llama-3_2-1b-instruct).
Ensure that you mount the directory containing the checkpoint when starting the container.
In this tutorial, we assume that the checkpoint is accessible under the path `"/checkpoints/llama-3_2-1b-instruct_v2.0"`.

> Note: You can learn more about deployment and available server endpoints from the ["Evaluating a NeMo checkpoint with lm-eval"](mmlu.ipynb) tutorial. 

In [ ]:
# modify this variable to point to your checkpoint
CHECKPOINT_PATH = "/checkpoints/llama-3_2-1b-instruct_v2.0"

# modify this path if you would like to save results in a different directory
WORKSPACE = "."

In [ ]:
deploy_process = subprocess.Popen(
    ["python", "./deploy.py", CHECKPOINT_PATH],
)

## 3. Evaluating the chat endpoint on HumanEval

Once the model is deployed, we can evaluate it.

Simple-evals provides a "chat" variant of the HumanEval benchmark, so we will send queries to the URL `"http://0.0.0.0:8080/v1/chat/completions/"`.

To learn more about the difference between "completions" and "chat" benchmarks, see the tutorial on ["Evaluating a NeMo checkpoint with lm-eval"](mmlu.ipynb).

In [ ]:
model_name = "megatron_model"
chat_url = "http://0.0.0.0:8080/v1/chat/completions/"

target_config = EvaluationTarget(
    api_endpoint=ApiEndpoint(url=chat_url, type=EndpointType.CHAT, model_id=model_name)
)
eval_config = EvaluationConfig(
    type="humaneval",
    output_dir=f"{WORKSPACE}/humaneval-results",
)
results = evaluate(target_cfg=target_config, eval_cfg=eval_config)

When the job finishes, we can close the server and inspect the results.

In [ ]:
deploy_process.send_signal(signal.SIGKILL)

In [ ]:
results

We can also examine the artifacts produced by the evaluation job:

In [ ]:
! ls {WORKSPACE}/humaneval-results

Inside the output directory, you can find a detailed report in HTML format: [humaneval.html](humaneval-results/humaneval.html).

This report contains a summary of the metrics as well as the input-output pairs for all samples used in the evaluation.